In [362]:
%matplotlib inline

In [363]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision
import math
from torch.autograd import Variable
import copy
import torchvision.transforms as transforms

In [364]:
import pandas as pd 
import numpy as np

In [365]:
rating_dir = './RCdata/rating_final.csv'

In [212]:
rest_deets_dir = './RCdata/geoplaces2.csv'

In [281]:
rest_cuisine_dir = './RCdata/chefmozcuisine.csv'

In [524]:
ratings = pd.read_csv(rating_dir)

In [407]:
ratings.rating.value_counts(normalize  = True)

2    0.418605
1    0.362618
0    0.218777
Name: rating, dtype: float64

In [529]:
ratings.rating.mean()

1.1998277347114557

In [530]:
ratings.rating.median()

1.0

In [214]:
rest_deets = pd.read_csv(rest_deets_dir, encoding = 'cp1252')

In [282]:
rest_cuisine = pd.read_csv(rest_cuisine_dir, encoding = 'cp1252')

In [295]:
x = rest_cuisine.loc[rest_cuisine['placeID'] == 135102]
hi = x['Rcuisine']
we = ''
for i in hi:
    we = we + i

In [219]:
rest_deets.loc[rest_deets['placeID'] == 134999]['name'][0]

'Kiku Cuernavaca'

In [367]:
from sklearn.model_selection import train_test_split
rating_train, rating_test = train_test_split(ratings, test_size=0.3)

In [525]:
n_users = len(ratings.userID.unique())
n_items = len(ratings.placeID.unique())

In [527]:
n_users
n_items

130

In [370]:
user_list = np.array(ratings.userID.unique())
item_list = np.array(ratings.placeID.unique())

In [ ]:
#use ratings = food + service rating put together

In [17]:
ratings_mat = np.zeros((n_users, n_items))
food_mat = np.zeros((n_users, n_items))
service_mat = np.zeros((n_users, n_items))

In [15]:
#ratings_mat = ratings_mat.tolist()

In [16]:
#ratings_mat.dim

In [31]:
ratings['userID'].iloc[0]

'U1077'

In [536]:
interactions_full[1].nnz

5

In [538]:
count = 0
for i in range(interactions_full.shape[0]):
    count = count + interactions_full[i].nnz

In [539]:
count/interactions_full.shape[0]

6.8768115942028984

In [182]:
for i in range(len(ratings)):
    user, = np.where(users == ratings['userID'].iloc[i])
    item, = np.where(items == ratings['placeID'].iloc[i])
    ratings_mat[user[0]][item[0]] = ratings['food_rating'].iloc[i] + ratings['service_rating'].iloc[i]
    #food_mat[user[0]][item[0]] = ratings['food_rating'].iloc[i]
    #service_mat[user[0]][item[0]] = ratings['service_rating'].iloc[i]

In [371]:
from scipy.sparse import rand as sprand
from scipy.sparse import lil_matrix

In [436]:
interactions_full = lil_matrix((n_users, n_items), dtype = float)
for row in ratings.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_full[user[0], item[0]] = row[4] + row[5]                  

In [435]:
interactions_full_orig = lil_matrix((n_users, n_items), dtype = float)
for row in ratings.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_full_orig[user[0], item[0]] = row[3]             

In [185]:
interactions_full

<138x130 sparse matrix of type '<class 'numpy.float64'>'
	with 949 stored elements in LInked List format>

In [437]:
interactions_train = lil_matrix((n_users, n_items), dtype = float)
for row in rating_train.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_train[user[0], item[0]] = row[4] + row[5]                  

In [438]:
interactions_train_orig = lil_matrix((n_users, n_items), dtype = float)
for row in rating_train.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_train_orig[user[0], item[0]] = row[3]

In [375]:
interactions_test = lil_matrix((n_users, n_items), dtype = float)
for row in rating_test.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_test[user[0], item[0]] = row[4] + row[5] 

In [502]:
interactions_test_orig = lil_matrix((n_users, n_items), dtype = float)
for row in rating_test.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_test_orig[user[0], item[0]] = row[3]

In [ ]:
#only use the ratings right now

In [376]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items,n_factors,sparse=False)
                                               

    # For convenience when we want to predict a sinble user-item pair.
    def predict(self, user, item):
        # Need to fit bias factors
        return (self.user_factors(user) * self.item_factors(item)).sum(1)
    
    # Much more efficient batch operator. This should be used for training purposes
    
    def forward(self, users, items):
        #return (self.user_factors(user) * self.item_factors(item)).sum(1)
    
        return torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))

In [506]:
class MatrixFactorizationClassifer(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items,n_factors,sparse=False)
        
    
    # For convenience when we want to predict a sinble user-item pair.
    def predict(self, user, item):
        # Need to fit bias factors
        pred = (self.user_factors(user) * self.item_factors(item)).sum(1)
        
        val = pred.data[0]
        
        if val <= np.percentile(x,22):
            val = 0
        elif np.percentile(x,22) < val < np.percentile(x, 58):
            val = 1
        elif val >= np.percentile(x, 58):
            val = 2
        
        print (val)
        final = torch.FloatTensor(val)
        return Variable(final)
    
    def predict_rating(self, user, item):
        return  (self.user_factors(user) * self.item_factors(item)).sum(1)
    
    def forward_ratings(self, users, items):
        pred = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        return pred
    
    def forward(self, users, items):
        #return (self.user_factors(user) * self.item_factors(item)).sum(1)
        pred = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        x = pred.data.numpy()
        
        for i in range(len(x)):
            for j in range(x.shape[1]):
                val = x[i][j]
                if val <= np.percentile(x,22):
                    x[i][j] = 0.0
                elif np.percentile(x,22) < val < np.percentile(x, 58):
                    x[i][j] = 1.0
                elif val >= np.percentile(x, 58):
                    x[i][j] = 2.0
        
        classes = torch.from_numpy(x)
        
        
    
        return Variable(classes,requires_grad=True)

In [349]:
class BiasedMatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items,mean, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors,sparse=False)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse = False)
        
        self.item_biases = torch.nn.Embedding(n_items,1, sparse = False)
                                               
        self.mu = mean
    
    #def forward(self, user, item):
        #pred = self.user_biases(user) + self.item_biases(item)
        #pred += (self.user_factors(user) * self.item_factors(item)).sum(1)
        #return pred
        
    def predict(self, user, item):
        pred = self.mu + self.user_biases(user) + self.item_biases(item)
        orig = (self.user_factors(user) * self.item_factors(item)).sum(1)
        final = pred + orig
        return final
    
    def forward(self, users, items):
        
        item_means = []
        user_means = []
        for i in range(len(users)):
            item_means.append(torch.t(self.item_biases(items)))
            
        for j in range(len(items)):
            user_means.append(torch.t(self.user_biases(users)))
        
        it_means = torch.cat(item_means, 0)
        us_means = torch.cat(user_means, 0)
        us_means = torch.t(us_means)
        
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        y = torch.add(orig, it_means)
        
        f = torch.add(y,us_means)
        
        result = torch.add(f,self.mu)
        
        return result

In [517]:
class BiasedMatrixFactorizationClassifier(torch.nn.Module):
    
    def __init__(self, n_users, n_items,mean, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors,sparse=False)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse = False)
        
        self.item_biases = torch.nn.Embedding(n_items,1, sparse = False)
                                               
        self.mu = mean
    
        
    def predict_rating(self, user, item):
        pred = self.mu + self.user_biases(user) + self.item_biases(item)
        orig = (self.user_factors(user) * self.item_factors(item)).sum(1)
        final = pred + orig
        return final
    
    #def predict()
    
    def forward_ratings(self, users, items):
        
        item_means = []
        user_means = []
        for i in range(len(users)):
            item_means.append(torch.t(self.item_biases(items)))
            
        for j in range(len(items)):
            user_means.append(torch.t(self.user_biases(users)))
        
        it_means = torch.cat(item_means, 0)
        us_means = torch.cat(user_means, 0)
        us_means = torch.t(us_means)
        
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        y = torch.add(orig, it_means)
        
        f = torch.add(y,us_means)
        
        result = torch.add(f,self.mu)
        
        return result
    
    def forward(self, users, items):
        
        item_means = []
        user_means = []
        for i in range(len(users)):
            item_means.append(torch.t(self.item_biases(items)))
            
        for j in range(len(items)):
            user_means.append(torch.t(self.user_biases(users)))
        
        it_means = torch.cat(item_means, 0)
        us_means = torch.cat(user_means, 0)
        us_means = torch.t(us_means)
        
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        y = torch.add(orig, it_means)
        
        f = torch.add(y,us_means)
        
        result = torch.add(f,self.mu)
        
        x = result.data.numpy()
        
        for i in range(len(x)):
            for j in range(x.shape[1]):
                val = x[i][j]
                if val <= np.percentile(x,22):
                    x[i][j] = 0.0
                elif np.percentile(x,22) < val < np.percentile(x, 58):
                    x[i][j] = 1.0
                elif val >= np.percentile(x, 58):
                    x[i][j] = 2.0
        
        classes = torch.from_numpy(x)
       
        return Variable(classes,requires_grad=True)
        

In [25]:
def get_batch(batch_size,ratings):
    # Sort our data and scramble it
    rows, cols = ratings.shape
    p = np.random.permutation(rows)
    
    # create batches
    sindex = 0
    eindex = batch_size
    while eindex < rows:
        batch = p[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch

    if eindex >= rows:
        batch = range(sindex,rows)
        yield batch

In [156]:
def full_batch(iterable):
    l = iterable.shape[0]
    #for ndx in range(0, l, n):
    print (l)
    yield iterable[0:l]

In [159]:
range(138)

range(130, 138)


In [301]:
def load_model(model,load_path):
    load_dict = torch.load(load_path)
    val_loss = load_dict['val_loss']
    model.load_state_dict(load_dict['model_state_dict'])


def checkpoint_model(val_loss, model,save_path):
    save_dict = dict(
                     val_loss=val_loss,
                     model_state_dict=model.state_dict())
                     #opt_state_dict=predictions.state_dict())
    torch.save(save_dict, save_path)

In [500]:
def run_validation(model, test_ratings, BATCH_SIZE, load_path):
    load_model(model, load_path)
    running_loss = 0.0
    loss_func = torch.nn.MSELoss()
    accur = 0
    for i,batch in enumerate(get_batch(BATCH_SIZE, test_ratings)):
        
        interactions = Variable(torch.FloatTensor(test_ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(test_ratings.shape[1])))
        
        predictions = model(rows, cols)
        loss = loss_func(predictions, interactions)
        
        running_loss += np.sqrt(loss.data[0])*BATCH_SIZE
        
        correct = get_accuracy(test_ratings, model, rows)
        accur = accur+ correct
    
    epoch_loss = running_loss/test_ratings.shape[0]
    return epoch_loss,accur

In [501]:
def run_validation_hi(model, test_ratings, BATCH_SIZE, load_path):
    load_model(model, load_path)
    running_loss = 0.0
    loss_func = torch.nn.MSELoss()
    accur = 0
    for i,batch in enumerate(get_batch(BATCH_SIZE, test_ratings)):
        
        interactions = Variable(torch.FloatTensor(test_ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(test_ratings.shape[1])))
        
        predictions = model(rows, cols)
        loss = loss_func(predictions, interactions)
        
        running_loss += np.sqrt(loss.data[0])*BATCH_SIZE
        
        correct = get_accuracy_hi(test_ratings, model, rows)
        accur = accur+ correct
    
    epoch_loss = running_loss/test_ratings.shape[0]
    return epoch_loss,accur

In [379]:
def get_accuracy(interactions_orig,model,rows):
    checks = np.nonzero(interactions_train)
    check_these = checks[0]
    correct = 0
    
    for u in range(len(rows)):
        
            user = rows.data[u]
            row = Variable(torch.LongTensor([np.long(user)]))
            where = np.where(check_these==user)
            index = where[0]
            if len(index) > 0:
                for c in range(len(index)):
                    rest = checks[1][index[c]]
                    col = Variable(torch.LongTensor([np.long(rest)]))
                    prediction = model.predict(row,col)
                   
                    val = prediction.data[0]
            
                    #val = np.around(get)
                    if val == interactions_train[user,rest]:
                        correct = correct + 1 
            else:
                next
    return correct

In [543]:
interactions_train[0,1]

0.0

In [547]:
def get_accuracy_test(interactions_orig,model,rows):
    
    
    checks = np.nonzero(interactions_orig)
    check_these = checks[0]
    
    batch = range(interactions_train.shape[0])
    interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
    users = Variable(torch.LongTensor(batch))
    cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
    predictions = model(users, cols)
    
    x = predictions.data.numpy()
    
    correct = 0
    
    for u in range(len(rows)):
        user = rows.data[u]
        where = np.where(check_these==user)
        index = where[0]
        if len(index) > 0:
            for c in range(len(index)):
                rest = checks[1][index[c]]
                prediction = predictions[u][rest]
                   
                val = prediction.data[0]
                
                if val <= np.percentile(x,22):
                    this_val = 0.0
                elif np.percentile(x,22) < val < np.percentile(x, 58):
                    this_val = 1.0
                elif val >= np.percentile(x, 58):
                     this_val= 2.0
        
                
                if this_val == interactions_orig[user,rest]:
                    correct = correct + 1 
            else:
                next
    return correct

In [497]:
def get_accuracy_hi(interactions_train, model, rows):
    checks = np.nonzero(interactions_train)
    check_these = checks[0]
    correct = 0
    
    #batch = range(interactions_train.shape[0])
    #interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
    #users = Variable(torch.LongTensor(rows))
    cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
    predictions = model(rows, cols)
    
    for u in range(len(rows)):
        user = rows.data[u]
        where = np.where(check_these==user)
        index = where[0]
        if len(index) > 0:
            for c in range(len(index)):
                rest = checks[1][index[c]]
                prediction = predictions[u][rest]
                   
                val = prediction.data[0]
                if val == interactions_train[user,rest]:
                    correct = correct + 1 
            else:
                next
    return correct

In [384]:
def generate_predictions(model, interactions_full, load_path):
    load_model(model, load_path)
    #final_top5 = [0 for i in range(5)]
    
    #for i,batch in enumerate(range(interactions_full.shape[0])):
    batch = range(interactions_full.shape[0])
    interactions = Variable(torch.FloatTensor(interactions_full[batch,:].toarray()))
    rows = Variable(torch.LongTensor(batch))
    cols = Variable(torch.LongTensor(np.arange(interactions_full.shape[1])))
    predictions = model(rows, cols)
    #predictions[interactions!=0] = 0
        
    val, index = torch.sort(predictions,1, descending = True)
        
    top5 = index[:,:5]
        
    return top5, val

In [507]:
def generate_predictions_hi(model, interactions_full, load_path):
    load_model(model, load_path)
    #final_top5 = [0 for i in range(5)]
    
    #for i,batch in enumerate(range(interactions_full.shape[0])):
    batch = range(interactions_full.shape[0])
    interactions = Variable(torch.FloatTensor(interactions_full[batch,:].toarray()))
    rows = Variable(torch.LongTensor(batch))
    cols = Variable(torch.LongTensor(np.arange(interactions_full.shape[1])))
    predictions = model.forward_ratings(rows, cols)
    predictions[interactions!=0] = 0
        
    val, index = torch.sort(predictions,1, descending = True)
        
    top5 = index[:,:5]
        
    return top5, val

In [243]:
def give_top5(recs, name_list, this_user, user_list, rest_list):
    
    user, = np.where(user_list == this_user)
    your_rests  = []
    for i in range(5):
        this_rest = recs[user[0]][i].data[0]
        rest = rest_list[this_rest]
        x = name_list.loc[name_list['placeID']== rest]
        if len(x) == 1:
            name = x['name'].item()
            your_rests.append(name)
        else:
            your_rests.append(rest)
    return your_rests

In [320]:
def give_top5_cuisines(recs, name_list, cuisine_list, user_list, rest_list):
    your_rests = [[0 for j in range(11)] for u in range(len(user_list))]
    for i in range(len(user_list)):
        user, = np.where(user_list == user_list[i])
        your_rests[i][0] = user_list[i]
        count = 1
        for r in range(5):
            this_rest = recs[user[0]][r].data[0]
            rest = rest_list[this_rest]
            x = name_list.loc[name_list['placeID'] == rest]
            name = x['name'].item()
            
            your_rests[i][count] = name
            count = count +1
            
            y = cuisine_list.loc[cuisine_list['placeID'] == rest]['Rcuisine']
            
            cuisine_types = ''
            for m in y:
                cuisine_types = cuisine_types + m + ', '
            
            your_rests[i][count] = cuisine_types
            count = count + 1
            
    return your_rests

In [328]:
batch_size = 10
save_path = './project_models/model.ckpt'
save_path_val = './project_models/modelv.ckpt'
factors = [2,3,4,5]
decays = [0.1,0.01,0.001]

In [247]:
for i in range(len(user_list)):
    print ("User: " + str(user_list[i]))
    results = give_top5(recs,rest_deets,user_list[i], user_list, item_list)
    print (results)

User: U1077
['Restaurant los Compadres', 'Restaurante la Estrella de Dima', 'TACOS CORRECAMINOS', 'Restaurant Oriental Express', 'El Rincon de San Francisco']
User: U1068
['Cafe Chaires', 'la Cochinita Pibil Restaurante Yucateco', 'Restaurant la Chalita', 'Restaurante y Pescaderia Tampico', 'Gorditas Dona Tota']
User: U1067
['carnitas mata calle Emilio Portes Gil', 'carnitas_mata', 'Hamburguesas Valle Dorado', 'Dominos Pizza', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1103
['Cafe Chaires', 'Restaurant la Chalita', 'Restaurant los Pinos', 'Mariscos Tia Licha', 'la Cochinita Pibil Restaurante Yucateco']
User: U1107
['Dominos Pizza', 'carnitas_mata', 'la Cochinita Pibil Restaurante Yucateco', 'Hamburguesas Valle Dorado', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1044
['El Rincon de San Francisco', 'Restaurant Orizatlan', 'TACOS CORRECAMINOS', 'Log Yin', 'la parroquia']
User: U1070
['Hamburguesas Valle Dorado', 'vips', 'don burguers', 'VIPS', 'el pueblito']
User: U1031
['Restaurant

['Restaurant los Compadres', 'Restaurante la Estrella de Dima', 'sirloin stockade', 'Tortas Locas Hipocampo', 'vips']
User: U1052
['carnitas_mata', 'carnitas mata calle Emilio Portes Gil', 'la Cochinita Pibil Restaurante Yucateco', 'Cafe Chaires', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1036
['vips', 'sirloin stockade', 'Restaurant los Compadres', 'don burguers', 'puesto de gorditas']
User: U1038
['Cafe Chaires', 'la Cochinita Pibil Restaurante Yucateco', 'Restaurant la Chalita', 'Restaurante y Pescaderia Tampico', 'Sushi Itto']
User: U1071
['vips', 'sirloin stockade', 'don burguers', 'Hamburguesas Valle Dorado', 'puesto de gorditas']
User: U1114
['Dominos Pizza', 'carnitas mata calle Emilio Portes Gil', 'carnitas_mata', 'Hamburguesas Valle Dorado', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1084
['vips', 'sirloin stockade', 'don burguers', 'Restaurant los Compadres', 'puesto de gorditas']
User: U1049
['Dominos Pizza', 'carnitas_mata', 'la Cochinita Pibil Restaurante Yucateco'

In [329]:
#Determine the best parameters for the model
best_val_loss = 1000.0
checks = np.nonzero(interactions_train)
checks_v = np.nonzero(interactions_test)
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    for r in range(len(decays)):
        print ("Decay: " + str(decays[r]))
        #best_val_loss = 1000.0
        best_train_loss = 1000.0
        accur = 0
        model = MatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], n_factors=factors[f])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=decays[r])
        for i in range(10):
            running_loss = 0.0
            accur = 0
            for m, batch in enumerate(get_batch(batch_size, interactions_train)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
                predictions = model(rows, cols)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
                
                correct = get_accuracy(interactions_train, model, rows)
                accur = accur + correct
                
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            
            print ('accuracy:' + str(accur/len(checks[0])))
            
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path)
            val_loss, val_accuracy = run_validation(model, interactions_test, batch_size, save_path)
            print ('val loss: ' + str(val_loss))
            print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print ('checkpointing_model with val loss: ' + str(best_val_loss))
                checkpoint_model(val_loss, model, save_path_val)

FACTOR: 2
Decay: 0.1
train_loss: 1.6268696338550017
accuracy:0.0475482912332838
checkpoint model with train loss: 1.6268696338550017
val loss: 1.5274177597456982
val accuracy: 0.03985507246376811
checkpointing_model with val loss: 1.5274177597456982
train_loss: 1.6366227392317987
accuracy:0.04309063893016345
val loss: 1.5283085542139683
val accuracy: 0.03985507246376811
train_loss: 1.6094827113307635
accuracy:0.0475482912332838
checkpoint model with train loss: 1.6094827113307635
val loss: 1.5595412687403851
val accuracy: 0.03985507246376811
train_loss: 1.615772105107701
accuracy:0.04903417533432392
val loss: 1.5484209493975511
val accuracy: 0.025362318840579712
train_loss: 1.6087360486610118
accuracy:0.04606240713224369
checkpoint model with train loss: 1.6087360486610118
val loss: 1.5528454653845671
val accuracy: 0.03985507246376811
train_loss: 1.571991368846112
accuracy:0.04606240713224369
checkpoint model with train loss: 1.571991368846112
val loss: 1.5474523541806449
val accuracy:

train_loss: 1.7971895851247208
accuracy:0.06983655274888559
val loss: 1.7275394871832246
val accuracy: 0.07971014492753623
FACTOR: 4
Decay: 0.1
train_loss: 2.051200363434256
accuracy:0.05943536404160475
checkpoint model with train loss: 2.051200363434256
val loss: 2.0069094339881817
val accuracy: 0.04710144927536232
train_loss: 2.0732898675061335
accuracy:0.06092124814264487
val loss: 2.0296215630672787
val accuracy: 0.050724637681159424
train_loss: 2.0476039420119587
accuracy:0.06240713224368499
checkpoint model with train loss: 2.0476039420119587
val loss: 2.020689114267295
val accuracy: 0.050724637681159424
train_loss: 2.075415669995724
accuracy:0.05943536404160475
val loss: 2.0426553856620715
val accuracy: 0.05434782608695652
train_loss: 2.053722127077762
accuracy:0.06092124814264487
val loss: 2.0360439778192534
val accuracy: 0.05434782608695652
train_loss: 2.076238699976724
accuracy:0.05794947994056464
val loss: 2.004694501890549
val accuracy: 0.04710144927536232
train_loss: 2.054

train_loss: 2.2671399574491593
accuracy:0.06686478454680535
val loss: 2.204077899201955
val accuracy: 0.07971014492753623
train_loss: 2.267897906094399
accuracy:0.07429420505200594
val loss: 2.2055492697782437
val accuracy: 0.07971014492753623
train_loss: 2.2768988217983495
accuracy:0.06983655274888559
val loss: 2.209645145434215
val accuracy: 0.07608695652173914


In [549]:
#BEST_MODEL
#attempt with the food + service ratings
#factors = 2, weight_decay = 0.1
best_val_loss = 1000.0
best_train_loss = 1000.0
accur = 0
model = MatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], n_factors=2)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)
for i in range(5):
    print ("Epoch: " + str(i))
    running_loss = 0.0
    accur = 0
    for m, batch in enumerate(get_batch(batch_size, interactions_train)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
        predictions = model(rows, cols)
        
        loss = loss_func(predictions, interactions)
        running_loss += np.sqrt(loss.data[0])*batch_size
                
        correct = get_accuracy_test(interactions_train_orig, model, rows)
        accur = accur + correct
                
        loss.backward()
    reg_loss_func.step()
    
    epoch_loss = running_loss/interactions_train.shape[0]
    print ('train_loss: ' + str(epoch_loss))
            
    print ('accuracy:' + str(accur/len(checks[0])))
            
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path)
        
    val_loss, predictions_z = run_validation(model, interactions_test, batch_size, save_path)
    print ('val loss: ' + str(val_loss))
    #print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

Epoch: 0
train_loss: 1.7445268493952018
accuracy:0.37147102526002973
checkpoint model with train loss: 1.7445268493952018
val loss: 1.6982276388930861
checkpointing_model with val loss: 1.6982276388930861
Epoch: 1
train_loss: 1.735576573111361
accuracy:0.3818722139673105
checkpoint model with train loss: 1.735576573111361
val loss: 1.6934513079351625
checkpointing_model with val loss: 1.6934513079351625
Epoch: 2
train_loss: 1.7343916919793785
accuracy:0.3491827637444279
checkpoint model with train loss: 1.7343916919793785
val loss: 1.6994631845794426
Epoch: 3
train_loss: 1.758359051601475
accuracy:0.32689450222882616
val loss: 1.7123278016402246
Epoch: 4
train_loss: 1.763514859237304
accuracy:0.3699851411589896
val loss: 1.7226251666966204


In [550]:
#BEST_MODEL
#factors = 2, weight_decay = 0.1
best_val_loss = 1000.0
best_train_loss = 1000.0
accur = 0
model = MatrixFactorization(interactions_train_orig.shape[0], interactions_train_orig.shape[1], n_factors=2)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)
for i in range(5):
    print ("Epoch: " + str(i))
    running_loss = 0.0
    accur = 0
    for m, batch in enumerate(get_batch(batch_size, interactions_train_orig)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train_orig.shape[1])))
        predictions = model(rows, cols)
        
        loss = loss_func(predictions, interactions)
        running_loss += np.sqrt(loss.data[0])*batch_size
                
        correct = get_accuracy(interactions_train_orig, model, rows)
        accur = accur + correct
                
        loss.backward()
    reg_loss_func.step()
    
    epoch_loss = running_loss/interactions_train.shape[0]
    print ('train_loss: ' + str(epoch_loss))
            
    print ('accuracy:' + str(accur/len(checks[0])))
            
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path)
        
    val_loss, predictions_z = run_validation(model, interactions_test, batch_size, save_path)
    print ('val loss: ' + str(val_loss))
    #print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

Epoch: 0
train_loss: 1.5707270398819881
accuracy:0.12184249628528974
checkpoint model with train loss: 1.5707270398819881
val loss: 1.5452841547323275
checkpointing_model with val loss: 1.5452841547323275
Epoch: 1
train_loss: 1.5618592088757617
accuracy:0.12184249628528974
checkpoint model with train loss: 1.5618592088757617
val loss: 1.5691157060045233
Epoch: 2
train_loss: 1.5406826376100953
accuracy:0.1188707280832095
checkpoint model with train loss: 1.5406826376100953
val loss: 1.58310896470134
Epoch: 3
train_loss: 1.5306873860436059
accuracy:0.11589895988112928
checkpoint model with train loss: 1.5306873860436059
val loss: 1.567590201984202
Epoch: 4
train_loss: 1.5397960368550259
accuracy:0.11292719167904904
val loss: 1.5665029607800622


In [542]:
best_val_loss = 1000.0
best_train_loss = 1000.0
accur = 0
model = MatrixFactorizationClassifer(interactions_train.shape[0], interactions_train.shape[1], n_factors=2)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)
for i in range(5):
    print ("Epoch: " + str(i))
    running_loss = 0.0
    accur = 0
    for m, batch in enumerate(get_batch(batch_size, interactions_train)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
        interactions2 = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
        predictions = model(rows, cols)
        
        #modeling 
        loss = loss_func(predictions, interactions)
        running_loss += np.sqrt(loss.data[0])*batch_size
                
        correct = get_accuracy_hi(interactions_train_orig, model, rows)
        accur = accur + correct
                
        loss.backward()
    reg_loss_func.step()
    
    #accur = get_accuracy_hi(interactions_train_orig, model)
    #print (accur)
    epoch_loss = running_loss/interactions_train.shape[0]
    print ('train_loss: ' + str(epoch_loss))
            
    print ('accuracy:' + str(accur/len(checks[0])))
            
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path)
        
    val_loss, val_accuracy = run_validation_hi(model, interactions_test_orig, batch_size, save_path)
    print ('val loss: ' + str(val_loss))
    print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

Epoch: 0
Variable containing:
  22
  77
 125
  32
  19
  89
  96
  76
 137
  68
[torch.LongTensor of size 10]

Variable containing:
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
  20
  21
  22
  23
  24
  25
  26
  27
  28
  29
  30
  31
  32
  33
  34
  35
  36
  37
  38
  39
  40
  41
  42
  43
  44
  45
  46
  47
  48
  49
  50
  51
  52
  53
  54
  55
  56
  57
  58
  59
  60
  61
  62
  63
  64
  65
  66
  67
  68
  69
  70
  71
  72
  73
  74
  75
  76
  77
  78
  79
  80
  81
  82
  83
  84
  85
  86
  87
  88
  89
  90
  91
  92
  93
  94
  95
  96
  97
  98
  99
 100
 101
 102
 103
 104
 105
 106
 107
 108
 109
 110
 111
 112
 113
 114
 115
 116
 117
 118
 119
 120
 121
 122
 123
 124
 125
 126
 127
 128
 129
[torch.LongTensor of size 130]

Variable containing:
 102
 101
  98
 131
  83
 130
  71
  35
  74
  82
[torch.LongTensor of size 10]

Variable containing:
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  1

KeyboardInterrupt: 

In [509]:
recs, predicts = generate_predictions_hi(model, interactions_full, save_path_val)

In [511]:
results = give_top5_cuisines(recs, rest_deets, rest_cuisine, user_list, item_list)

In [513]:
df = pd.DataFrame(results,columns = col_names)

In [514]:
df

,User,Rec1,Rec1_Cuisines,Rec2,Rec2_Cuisines,Rec3,Rec3_Cuisines,Rec4,Rec4_Cuisines,Rec5,Rec5_Cuisines
0,U1077,Sirlone,"International,",Restaurant los Compadres,,Restaurant Oriental Express,"Chinese,",Restaurante Bar Fu-hao,"Bar,",Cabana Huasteca,"Mexican,"
1,U1068,Restaurant los Compadres,,Sirlone,"International,",Restaurant Oriental Express,"Chinese,",Cabana Huasteca,"Mexican,",Restaurante Bar Fu-hao,"Bar,"
2,U1067,la Estrella de Dimas,,La Virreina,"Mexican,",cafe punta del cielo,"Cafeteria,",little pizza Emilio Portes Gil,"Armenian,",cafe ambar,
3,U1103,Restaurant los Compadres,,TACOS CORRECAMINOS,"Mexican,",la Estrella de Dimas,,Cabana Huasteca,"Mexican,",Restaurante Bar Fu-hao,"Bar,"
4,U1107,Restaurant los Compadres,,Sirlone,"International,",Restaurant Oriental Express,"Chinese,",Cabana Huasteca,"Mexican,",Restaurante Bar Fu-hao,"Bar,"
5,U1044,Restaurant los Compadres,,Cabana Huasteca,"Mexican,",Restaurante Bar Fu-hao,"Bar,",TACOS CORRECAMINOS,"Mexican,",la Estrella de Dimas,
6,U1070,puesto de gorditas,"Regional,",Restaurant Las Mananitas,"International,",Tortas y hamburguesas el gordo,"Burgers,",Restaurante la Parroquia Potosina,"Contemporary,",Restaurant Orizatlan,"Mexican,"
7,U1031,El Rincón de San Francisco,"Mexican,",Restaurante Tiberius,"Pizzeria,",Tortas y hamburguesas el gordo,"Burgers,",Potzocalli,,Restaurant Orizatlan,"Mexican,"
8,U1082,Sirlone,"International,",Restaurante la Parroquia Potosina,"Contemporary,",emilianos,"Bar_Pub_Brewery,",Restaurant Oriental Express,"Chinese,",Restaurant los Compadres,
9,U1023,El Rincón de San Francisco,"Mexican,",Restaurante Tiberius,"Pizzeria,",Potzocalli,,little pizza Emilio Portes Gil,"Armenian,",Restaurante Guerra,


In [331]:
recs = generate_predictions(model, interactions_full, save_path_val)
results = give_top5_cuisines(recs, rest_deets, rest_cuisine, user_list, item_list)

In [332]:
col_names = ['User','Rec1', 'Rec1_Cuisines', 'Rec2', 'Rec2_Cuisines', 'Rec3', 'Rec3_Cuisines', 'Rec4','Rec4_Cuisines',  'Rec5', 'Rec5_Cuisines']

In [333]:
df = pd.DataFrame(results,columns = col_names)

In [334]:
df

,User,Rec1,Rec1_Cuisines,Rec2,Rec2_Cuisines,Rec3,Rec3_Cuisines,Rec4,Rec4_Cuisines,Rec5,Rec5_Cuisines
0,U1077,Carreton de Flautas y Migadas,"Mexican,",Restaurant and Bar and Clothesline Carlos N Ch...,"Bar, Bar_Pub_Brewery,",emilianos,"Bar_Pub_Brewery,",La Posada del Virrey,"International,",Carls Jr,"Burgers,"
1,U1068,Cabana Huasteca,"Mexican,",Restaurante El Reyecito,"Fast_Food,",El cotorreo,"Family, Mexican,",Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,",Potzocalli,
2,U1067,Restaurante la Gran Via,,churchs,,Mcdonalds Parque Tangamanga,"Burgers, Fast_Food,",Kiku Cuernavaca,"Japanese,",la perica hamburguesa,"Italian,"
3,U1103,Cabana Huasteca,"Mexican,",Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,",cafe punta del cielo,"Cafeteria,",Restaurante El Reyecito,"Fast_Food,",Arrachela Grill,
4,U1107,Carls Jr,"Burgers,",crudalia,"Bar,",Carreton de Flautas y Migadas,"Mexican,",Restaurante la Cantina,"Bar, Bar_Pub_Brewery,",emilianos,"Bar_Pub_Brewery,"
5,U1044,Restaurante de Mariscos la Langosta,"Seafood,",TACOS EL GUERO,"Mexican,",El Rincón de San Francisco,"Mexican,",cafe punta del cielo,"Cafeteria,",La Cantina Restaurante,"Bar, Bar_Pub_Brewery,"
6,U1070,crudalia,"Bar,",Carls Jr,"Burgers,",Sanborns Casa Piedra,,Restaurante la Cantina,"Bar, Bar_Pub_Brewery,",Restaurante la Gran Via,
7,U1031,Carreton de Flautas y Migadas,"Mexican,",emilianos,"Bar_Pub_Brewery,",Carls Jr,"Burgers,",crudalia,"Bar,",Restaurante la Cantina,"Bar, Bar_Pub_Brewery,"
8,U1082,Restaurante de Mariscos la Langosta,"Seafood,",cafe punta del cielo,"Cafeteria,",TACOS EL GUERO,"Mexican,",Arrachela Grill,,La Cantina Restaurante,"Bar, Bar_Pub_Brewery,"
9,U1023,Restaurante la Gran Via,,churchs,,Kiku Cuernavaca,"Japanese,",Dominos Pizza,"Pizzeria,",Sanborns Casa Piedra,


In [337]:
#trying it without weight decay
#What feature works the best? 
best_val_loss = 1000.0
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    best_train_loss = 1000.0
    model = MatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], n_factors=factors[f])
    loss_func = torch.nn.MSELoss()
    reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6)
    for i in range(10):
        running_loss = 0.0
        accur = 0
        for m, batch in enumerate(get_batch(batch_size, interactions_train)):
            reg_loss_func.zero_grad()
            interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
            rows = Variable(torch.LongTensor(batch))
            cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
            predictions = model(rows, cols)
        
            loss = loss_func(predictions, interactions)
            running_loss += np.sqrt(loss.data[0])*batch_size
            
            correct = get_accuracy(interactions_train, model, rows)
            accur = accur + correct
        
            loss.backward()
        reg_loss_func.step()
        
        epoch_loss = running_loss/interactions_train.shape[0]
        print ('train_loss: ' + str(epoch_loss))
        print ('accuracy:' + str(accur/len(checks[0])))
        
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss
            print ('checkpoint model with train loss: ' + str(best_train_loss))
            checkpoint_model(epoch_loss, model, save_path)
        val_loss, val_accuracy = run_validation(model, interactions_test, batch_size, save_path)
    
        print ('val loss: ' + str(val_loss))
        print ('val accuracy:' + str(val_accuracy/len(checks_v[0])))
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print ('checkpointing_model with val loss: ' + str(best_val_loss))
            checkpoint_model(val_loss, model, save_path_val)

FACTOR: 2
train_loss: 1.5682497283603707
accuracy:0.06983655274888559
checkpoint model with train loss: 1.5682497283603707
val loss: 1.493789642536607
val accuracy:0.06159420289855073
checkpointing_model with val loss: 1.493789642536607
train_loss: 1.5352989589048383
accuracy:0.0713224368499257
checkpoint model with train loss: 1.5352989589048383
val loss: 1.4881491463649967
val accuracy:0.06521739130434782
checkpointing_model with val loss: 1.4881491463649967
train_loss: 1.5509188228942803
accuracy:0.0713224368499257
val loss: 1.4960720767494704
val accuracy:0.057971014492753624
train_loss: 1.5768431448153275
accuracy:0.07280832095096583
val loss: 1.484385432929435
val accuracy:0.06521739130434782
checkpointing_model with val loss: 1.484385432929435
train_loss: 1.5349212790058355
accuracy:0.06835066864784546
checkpoint model with train loss: 1.5349212790058355
val loss: 1.496869122407948
val accuracy:0.06159420289855073
train_loss: 1.566760721815084
accuracy:0.0713224368499257
val los

In [338]:
best_val_loss = 1000.0

best_train_loss = 1000.0
model = MatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], n_factors=2)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6)
for i in range(10):
    running_loss = 0.0
    accur = 0
    for m, batch in enumerate(get_batch(batch_size, interactions_train)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
        predictions = model(rows, cols)
        
        loss = loss_func(predictions, interactions)
        running_loss += np.sqrt(loss.data[0])*batch_size
            
        correct = get_accuracy(interactions_train, model, rows)
        accur = accur + correct
        
        loss.backward()
    reg_loss_func.step()
        
    epoch_loss = running_loss/interactions_train.shape[0]
    print ('train_loss: ' + str(epoch_loss))
    print ('accuracy:' + str(accur/len(checks[0])))
        
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path)
    val_loss, val_accuracy = run_validation(model, interactions_test, batch_size, save_path)
    
    print ('val loss: ' + str(val_loss))
    print ('val accuracy:' + str(val_accuracy/len(checks_v[0])))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

train_loss: 1.5379750308484619
accuracy:0.05052005943536404
checkpoint model with train loss: 1.5379750308484619
val loss: 1.4602353394521397
val accuracy:0.057971014492753624
checkpointing_model with val loss: 1.4602353394521397
train_loss: 1.5166856053326967
accuracy:0.05052005943536404
checkpoint model with train loss: 1.5166856053326967
val loss: 1.4379631874642136
val accuracy:0.06159420289855073
checkpointing_model with val loss: 1.4379631874642136
train_loss: 1.5311400419670478
accuracy:0.05200594353640416
val loss: 1.4564102242855939
val accuracy:0.06159420289855073
train_loss: 1.5370367736434676
accuracy:0.05349182763744428
val loss: 1.4602721155179939
val accuracy:0.06521739130434782
train_loss: 1.5378680279179904
accuracy:0.05200594353640416
val loss: 1.4770477917359834
val accuracy:0.057971014492753624
train_loss: 1.535450559094825
accuracy:0.05349182763744428
val loss: 1.4354603927978675
val accuracy:0.06521739130434782
checkpointing_model with val loss: 1.4354603927978675

In [339]:
recs = generate_predictions(model, interactions_full, save_path_val)
results = give_top5_cuisines(recs, rest_deets, rest_cuisine, user_list, item_list)
df1 = pd.DataFrame(results,columns = col_names)

In [340]:
df1

,User,Rec1,Rec1_Cuisines,Rec2,Rec2_Cuisines,Rec3,Rec3_Cuisines,Rec4,Rec4_Cuisines,Rec5,Rec5_Cuisines
0,U1077,little pizza Emilio Portes Gil,"Armenian,",Restaurante 75,,Hamburguesas saul,"Cafeteria,",Rincon del Bife,,El Club,
1,U1068,Koye Sushi,"Chinese,",Tortas Locas Hipocampo,"Fast_Food,",shi ro ie,"Japanese,",puesto de gorditas,"Regional,",Vips,
2,U1067,la Cochinita Pibil Restaurante Yucateco,"Mexican,",Restaurante Marisco Sam,"Seafood,",shi ro ie,"Japanese,",los Toneles,,cafe punta del cielo,"Cafeteria,"
3,U1103,Hamburguesas saul,"Cafeteria,",Restaurant Wu Zhuo Yi,"Chinese,",la perica hamburguesa,"Italian,",Carreton de Flautas y Migadas,"Mexican,",puesto de gorditas,"Regional,"
4,U1107,El Club,,cafe punta del cielo,"Cafeteria,",vips,"Mexican,",Restaurante y Pescaderia Tampico,"Seafood,",Restaurante Marisco Sam,"Seafood,"
5,U1044,Koye Sushi,"Chinese,",Tortas Locas Hipocampo,"Fast_Food,",shi ro ie,"Japanese,",puesto de gorditas,"Regional,",Vips,
6,U1070,little pizza Emilio Portes Gil,"Armenian,",Hamburguesas saul,"Cafeteria,",Restaurante 75,,Rincon del Bife,,la perica hamburguesa,"Italian,"
7,U1031,Hamburguesas saul,"Cafeteria,",little pizza Emilio Portes Gil,"Armenian,",Restaurant Wu Zhuo Yi,"Chinese,",puesto de gorditas,"Regional,",Carreton de Flautas y Migadas,"Mexican,"
8,U1082,El Club,,vips,"Mexican,",cafe punta del cielo,"Cafeteria,",Restaurante y Pescaderia Tampico,"Seafood,",Restaurante 75,
9,U1023,El Club,,cafe punta del cielo,"Cafeteria,",vips,"Mexican,",Restaurante y Pescaderia Tampico,"Seafood,",Restaurante Bar Fu-hao,"Bar,"


In [521]:
save_path_b = './project_models/bmodel.ckpt'
save_path_val_b = './project_models/bmodelv.ckpt'

In [523]:
best_val_loss = 1000.0
best_train_loss = 1000.0
mean = np.mean(interactions_train)
accur = 0
model = BiasedMatrixFactorizationClassifier(interactions_train.shape[0], interactions_train.shape[1],mean, n_factors=2)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)
for i in range(8):
    print ("Epoch: " + str(i))
    running_loss = 0.0
    accur = 0
    for m, batch in enumerate(get_batch(batch_size, interactions_train)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
        interactions2 = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
        predictions = model(rows, cols)
        
        loss = loss_func(predictions, interactions2)
        running_loss += np.sqrt(loss.data[0])*batch_size
                
        correct = get_accuracy_hi(interactions_train_orig, model, rows)
        accur = accur + correct
                
        loss.backward()
    reg_loss_func.step()
    
    #accur = get_accuracy_hi(interactions_train_orig, model)
    #print (accur)
    epoch_loss = running_loss/interactions_train.shape[0]
    print ('train_loss: ' + str(epoch_loss))
            
    print ('accuracy:' + str(accur/len(checks[0])))
            
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path_b)
        
    val_loss, val_accuracy = run_validation_hi(model, interactions_test_orig, batch_size, save_path_b)
    print ('val loss: ' + str(val_loss))
    print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val_b)

Epoch: 0
train_loss: 1.268021623764264
accuracy:0.2689450222882615
checkpoint model with train loss: 1.268021623764264
val loss: 1.2684205658640317
val accuracy: 0.36231884057971014
checkpointing_model with val loss: 1.2684205658640317
Epoch: 1
train_loss: 1.2663894154073452
accuracy:0.2585438335809807
checkpoint model with train loss: 1.2663894154073452
val loss: 1.2650761637566013
val accuracy: 0.35144927536231885
checkpointing_model with val loss: 1.2650761637566013
Epoch: 2
train_loss: 1.2780589955968145
accuracy:0.26151560178306094
val loss: 1.267368896244749
val accuracy: 0.36594202898550726
Epoch: 3
train_loss: 1.2689088754383795
accuracy:0.274888558692422
val loss: 1.2595045695741836
val accuracy: 0.3333333333333333
checkpointing_model with val loss: 1.2595045695741836
Epoch: 4
train_loss: 1.2725201075994208
accuracy:0.2362555720653789
val loss: 1.2615301406907327
val accuracy: 0.3442028985507246
Epoch: 5
train_loss: 1.2650054610451558
accuracy:0.2540861812778603
checkpoint mod

In [361]:
#figure out how to get accuracy to work for here too
best_val_loss = 1000.0
mean = np.mean(interactions_train)
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    for r in range(len(decays)):
        print ("Decay: " + str(decays[r]))
        best_train_loss = 1000.0
        model = BiasedMatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], mean, n_factors=factors[f])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=decays[r])
        for i in range(10):
            running_loss = 0.0
            accur = 0
            for m, batch in enumerate(get_batch(batch_size, interactions_train)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
                predictions = model(rows, cols)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
                
                #correct = get_accuracy(interactions_train, model, rows)
                #accur = accur + correct
        
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            #print ('accuracy: ' + str(accur/len(checks[0])))
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path_b)
            val_loss, val_accuracy = run_validation(model, interactions_test, batch_size, save_path_b)
            print ('val loss: ' + str(val_loss))
            print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print ('checkpointing_model with val loss: ' + str(best_val_loss))
                checkpoint_model(val_loss, model, save_path_val_b)

FACTOR: 2
Decay: 0.1
train_loss: 2.0853790586394108
checkpoint model with train loss: 2.0853790586394108
val loss: 1.9903669498594996
val accuracy: 0.0
checkpointing_model with val loss: 1.9903669498594996
train_loss: 2.0723903014157656
checkpoint model with train loss: 2.0723903014157656
val loss: 2.0147439338446533
val accuracy: 0.0
train_loss: 2.0353521438565
checkpoint model with train loss: 2.0353521438565
val loss: 2.0229335985838364
val accuracy: 0.0
train_loss: 2.067931044106231
val loss: 2.0105523277896347
val accuracy: 0.0
train_loss: 2.057918243990295
val loss: 2.0203420808932937
val accuracy: 0.0
train_loss: 2.064630110047719
val loss: 2.039006490900604
val accuracy: 0.0
train_loss: 2.069370373584729
val loss: 2.029394090439562
val accuracy: 0.0
train_loss: 2.0363602812692374
val loss: 2.0298443420537393
val accuracy: 0.0
train_loss: 2.049759149584906
val loss: 2.017131575323352
val accuracy: 0.0
train_loss: 2.0666982643688074
val loss: 2.0386681529274338
val accuracy: 0.0


train_loss: 2.6660406424397265
val loss: 2.6513159019653676
val accuracy: 0.0
train_loss: 2.643468425267876
checkpoint model with train loss: 2.643468425267876
val loss: 2.619931647454562
val accuracy: 0.0
train_loss: 2.6515170756849455
val loss: 2.637985205585593
val accuracy: 0.0
train_loss: 2.6478512689334788
val loss: 2.645035833639765
val accuracy: 0.0
train_loss: 2.654229815527048
val loss: 2.6412653863525315
val accuracy: 0.0
train_loss: 2.6569145200679967
val loss: 2.6522497740552193
val accuracy: 0.0
train_loss: 2.676080511508304
val loss: 2.5948859116342438
val accuracy: 0.0
FACTOR: 5
Decay: 0.1
train_loss: 2.7318106253362004
checkpoint model with train loss: 2.7318106253362004
val loss: 2.6856935912928503
val accuracy: 0.0
train_loss: 2.7371240288064125
val loss: 2.685688866517505
val accuracy: 0.0
train_loss: 2.7295754911923047
checkpoint model with train loss: 2.7295754911923047
val loss: 2.7098560585459515
val accuracy: 0.0
train_loss: 2.7132936142123545
checkpoint model 

In [66]:
best_val_loss = 1000.0
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    best_train_loss = 1000.0
    model = BiasedMatrixFactorization(interactions_train.shape[0], interactions_train.shape[1],mean, n_factors=factors[f])
    loss_func = torch.nn.MSELoss()
    reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6)
    for i in range(10):
        running_loss = 0.0
        for m, batch in enumerate(get_batch(batch_size, interactions_train)):
            reg_loss_func.zero_grad()
            interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
            rows = Variable(torch.LongTensor(batch))
            cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
            predictions = model(rows, cols)
        
            loss = loss_func(predictions, interactions)
            running_loss += np.sqrt(loss.data[0])*batch_size
        
            loss.backward()
        reg_loss_func.step()
    
        epoch_loss = running_loss/interactions_train.shape[0]
        print ('train_loss: ' + str(epoch_loss))
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss
            print ('checkpoint model with train loss: ' + str(best_train_loss))
            checkpoint_model(epoch_loss, model, save_path)
    val_loss = run_validation(model, interactions_test, batch_size, save_path)
    print ('val loss: ' + str(val_loss))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

FACTOR: 2
train_loss: 2.0601148269819056
checkpoint model with train loss: 2.0601148269819056
train_loss: 2.078074929939537
train_loss: 2.071905343596753
train_loss: 2.075484468429301
train_loss: 2.092951358020045
train_loss: 2.0661176163857187
train_loss: 2.0865121634017103
train_loss: 2.0831317007630457
train_loss: 2.0904322533952024
train_loss: 2.0960979413224243
val loss: 2.0547025545466773
checkpointing_model with val loss: 2.0547025545466773
FACTOR: 3
train_loss: 2.4422406997496546
checkpoint model with train loss: 2.4422406997496546
train_loss: 2.436855214838995
checkpoint model with train loss: 2.436855214838995
train_loss: 2.427085590662223
checkpoint model with train loss: 2.427085590662223
train_loss: 2.4439370718014892
train_loss: 2.444650610989361
train_loss: 2.4265913143154667
checkpoint model with train loss: 2.4265913143154667
train_loss: 2.438623116704
train_loss: 2.4389246554422184
train_loss: 2.4368984165790972
train_loss: 2.417333068248689
checkpoint model with trai